# Sensor Grid Plots

## Imports and Global Helper Functions

In [ ]:
import json

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps, register_fausans_font
import biopsykit as bp

from stresspose_analysis.datasets.pilotstudy import PilotStudyDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

## Path Setup

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
output_path = Path("../../results/plots/sensor_grid_plot")
bp.utils.file_handling.mkdirs(output_path)

In [ ]:
dataset = PilotStudyDataset(base_path)

## Get Data

In [ ]:
mocap_data = dataset.get_subset(subject="VP_04", condition="tsst").mocap_data

## Sensor Grid Plot – Body Parts

In [ ]:
body_parts = [
    "Head",
    "Hips",
    "LeftArm",
    "LeftFoot",
    "LeftForeArm",
    "LeftHand",
    "LeftLeg",
    "LeftShoulder",
    "LeftUpLeg",
    "Neck",
    "RightArm",
    "RightFoot",
    "RightForeArm",
    "RightHand",
    "RightLeg",
    "RightShoulder",
    "RightUpLeg",
    "Spine",
    "Spine1",
    "Spine2",
]

data_plot = mocap_data.xs("calc", level="data_format", axis=1)
data_plot = data_plot.reindex(body_parts, level="body_part", axis=1)
data_plot = data_plot.xs("ang_vel", level="channel", axis=1)
data_plot = data_plot.loc[0:5]
data_plot = data_plot.rolling(5).mean().dropna()
data_plot.head()

In [ ]:
ncols = 5
nrows = 4

fig, axs = plt.subplots(figsize=(ncols, nrows), ncols=ncols, nrows=nrows, sharex=True)
axs = axs.flatten()

for (key, df), ax in zip(data_plot.groupby("body_part", axis=1), axs):
    df.plot(ax=ax, legend=False)
    ax.set_title(key, fontsize="smaller")
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.set_xlabel(None)

fig.tight_layout(h_pad=0.5, w_pad=0.5, pad=0.1)
fig.savefig(output_path.joinpath("img_sensor_grid_body_parts.pdf"), transparent=True)

## Sensor Grid Plot – Channels

In [ ]:
data_plot = mocap_data.xs("calc", level="data_format", axis=1)
data_plot = data_plot.xs("Head", level="body_part", axis=1)
data_plot = data_plot.rename(
    {"pos": "Position", "vel": "Velocity", "quat": "Quaternion", "acc": "Acceleration", "ang_vel": "Angular Vel."},
    axis=1,
    level="channel",
)
data_plot = data_plot.loc[0:5]
data_plot = data_plot.rolling(5).mean().dropna()
data_plot.head()

### Horizontal Plot

In [ ]:
ncols = 5
nrows = 1

fig, axs = plt.subplots(figsize=(ncols, nrows), ncols=ncols, nrows=nrows, sharex=True)
axs = axs.flatten()

for (key, df), ax in zip(data_plot.groupby("channel", axis=1), axs):
    df.plot(ax=ax, legend=False)
    ax.set_title(key, fontsize="smaller")
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.set_xlabel(None)

fig.tight_layout(h_pad=0.5, w_pad=0.5, pad=0.1)
fig.savefig(output_path.joinpath("img_sensor_grid_channels_vert.pdf"), transparent=True)

### Vertical Plot

In [ ]:
ncols = 1
nrows = 5

fig, axs = plt.subplots(figsize=(ncols, nrows), ncols=ncols, nrows=nrows, sharex=True)
axs = axs.flatten()

for (key, df), ax in zip(data_plot.groupby("channel", axis=1), axs):
    df.plot(ax=ax, legend=False)
    ax.set_title(key, fontsize="smaller")
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.set_xlabel(None)

fig.tight_layout(h_pad=0.5, w_pad=0.5, pad=0.1)
fig.savefig(output_path.joinpath("img_sensor_grid_channels_hor.pdf"), transparent=True)